In [11]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [12]:
x1 = pd.read_csv('C:/Users/FRANK/Desktop/Proyectos Linkedin/Python Proyectos/Dia 12 - Dia 13 - Random Forest/train.csv') # # Trabajar con este dataset 
x2 = pd.read_csv('C:/Users/FRANK/Desktop/Proyectos Linkedin/Python Proyectos/Dia 12 - Dia 13 - Random Forest/test.csv') # Utilizar la data de train.csv para hallar la data de test.csv
x1.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [13]:
print('Número de observaciones (filas) = {}'.format(x1.shape[0]))
print('Total de variables disponibles (columnas) = {} '.format(x1.columns.shape[0]))
print('Columnas que cuentan con un valor nulo -')
print(x1.isnull().sum())

Número de observaciones (filas) = 1460
Total de variables disponibles (columnas) = 81 
Columnas que cuentan con un valor nulo -
Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [14]:
x1 = x1.drop(columns='Id')

In [15]:
# Identificar y mostrar solo las columnas con valores nulos
print('Número de observaciones (filas) = {}'.format(x1.shape[0]))
print('Total de variables disponibles (columnas) = {} '.format(x1.columns.shape[0]))
null_columns = x1.isnull().sum()
columns_with_nulls = null_columns[null_columns > 0]
print('Columnas que cuentan con valores nulos y la cantidad de nulos en cada columna:')
print(columns_with_nulls)

Número de observaciones (filas) = 1460
Total de variables disponibles (columnas) = 80 
Columnas que cuentan con valores nulos y la cantidad de nulos en cada columna:
LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [16]:
# Identificar los tipos de columnas con valores nulos
null_columns_type = x1[columns_with_nulls.index].dtypes
print(null_columns_type)

LotFrontage     float64
Alley            object
MasVnrType       object
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
FireplaceQu      object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
PoolQC           object
Fence            object
MiscFeature      object
dtype: object


In [17]:
x1['Electrical'].value_counts()

Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [18]:
x1['Electrical'] = x1['Electrical'].fillna(x1['Electrical'].mode()[0])

In [19]:
mapping = {
    'BsmtQual': {'TA': 1, 'Gd': 2, 'Ex': 3, 'Fa': 4},
    'BsmtCond': {'TA': 1, 'Gd': 2, 'Fa': 3, 'Po': 4},
    'BsmtExposure': {'No': 1, 'Av': 2, 'Gd': 3, 'Mn': 4},
    'BsmtFinType1': {'Unf': 1, 'GLQ': 2, 'ALQ': 3, 'BLQ': 4, 'Rec': 5, 'LwQ': 6},
    'BsmtFinType2': {'Unf': 1, 'Rec': 2, 'LwQ': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'GarageType': {'Attchd': 1, 'Detchd': 2, 'BuiltIn': 3, 'Basment': 4, 'CarPort': 5, '2Types': 6},
    'GarageFinish': {'Fin': 1, 'RFn': 2, 'Unf': 3},
    'GarageQual': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5},
    'GarageCond': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5}
}

# Reemplazar los valores categóricos por sus equivalentes numéricos para las columnas iniciales
for col, map_dict in mapping.items():
    x1[col] = x1[col].map(map_dict)

# Inicializar el imputador KNN
imputer = KNNImputer(n_neighbors=5)

# Seleccionar las columnas que requieren imputación inicialmente
columns_to_impute_initial = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                             'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']

# Aplicar la imputación a las columnas iniciales
x1[columns_to_impute_initial] = imputer.fit_transform(x1[columns_to_impute_initial])

In [20]:
null_columns = x1.isnull().sum()
columns_with_nulls = null_columns[null_columns > 0]
print('Columnas que cuentan con valores nulos y la cantidad de nulos en cada columna:')
print(columns_with_nulls)

Columnas que cuentan con valores nulos y la cantidad de nulos en cada columna:
LotFrontage     259
Alley          1369
MasVnrType      872
MasVnrArea        8
FireplaceQu     690
PoolQC         1453
Fence          1179
MiscFeature    1406
dtype: int64


In [21]:
# Identificar los tipos de columnas con valores nulos
null_columns_type = x1[columns_with_nulls.index].dtypes
print(null_columns_type)

LotFrontage    float64
Alley           object
MasVnrType      object
MasVnrArea     float64
FireplaceQu     object
PoolQC          object
Fence           object
MiscFeature     object
dtype: object


In [22]:
x1[['LotFrontage','MasVnrArea']].corr()

,LotFrontage,MasVnrArea
LotFrontage,1.000000,0.193458
MasVnrArea,0.193458,1.000000


In [23]:
# Imputar valores nulos en características binarias usando KNN
knn_imputer = KNNImputer(n_neighbors=2)
x1[['LotFrontage','MasVnrArea']] = knn_imputer.fit_transform(x1[['LotFrontage','MasVnrArea']])
print("Después de imputar la columna binaria con KNN:")
print(x1)

Después de imputar la columna binaria con KNN:
      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0             60       RL         65.0     8450   Pave   NaN      Reg   
1             20       RL         80.0     9600   Pave   NaN      Reg   
2             60       RL         68.0    11250   Pave   NaN      IR1   
3             70       RL         60.0     9550   Pave   NaN      IR1   
4             60       RL         84.0    14260   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1455          60       RL         62.0     7917   Pave   NaN      Reg   
1456          20       RL         85.0    13175   Pave   NaN      Reg   
1457          70       RL         66.0     9042   Pave   NaN      Reg   
1458          20       RL         68.0     9717   Pave   NaN      Reg   
1459          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... PoolArea PoolQC  Fence MiscFeature

In [24]:
# Función para codificar variables categóricas y entrenar modelo Random Forest
def impute_with_random_forest_encoded(data, target_column, is_classification=False):
    # Separar filas completas y filas con datos faltantes
    complete_rows = data.dropna(subset=[target_column])
    incomplete_rows = data[data[target_column].isnull()]
    
    # Si no hay filas incompletas, regresar los datos sin cambios
    if incomplete_rows.empty:
        return data
    
    # Separar características y target
    X_train = complete_rows.drop(columns=[target_column])
    y_train = complete_rows[target_column]
    X_predict = incomplete_rows.drop(columns=[target_column])
    
    # Codificar variables categóricas
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_train_encoded = encoder.fit_transform(X_train.select_dtypes(include='object'))
    X_predict_encoded = encoder.transform(X_predict.select_dtypes(include='object'))
    
    # Concatenar columnas numéricas y codificadas
    X_train_encoded = np.hstack((X_train.select_dtypes(exclude='object').values, X_train_encoded))
    X_predict_encoded = np.hstack((X_predict.select_dtypes(exclude='object').values, X_predict_encoded))
    
    # Imputar temporalmente los valores NaN en X_train_encoded y X_predict_encoded
    imputer = SimpleImputer(strategy='mean')
    X_train_encoded = imputer.fit_transform(X_train_encoded)
    X_predict_encoded = imputer.transform(X_predict_encoded)
    
    # Seleccionar el modelo apropiado
    if is_classification:
        model = RandomForestClassifier()
    else:
        model = RandomForestRegressor()
    
    # Entrenar el modelo
    model.fit(X_train_encoded, y_train)
    
    # Predecir valores faltantes
    predicted_values = model.predict(X_predict_encoded)
    
    # Imputar valores predichos
    data.loc[data[target_column].isnull(), target_column] = predicted_values
    
    return data

# Supongamos que tienes un DataFrame llamado x1 y quieres imputar valores nulos
columns_with_nulls = x1.columns[x1.isnull().any()].tolist()
null_columns_types = x1[columns_with_nulls].dtypes

# Imputar valores nulos usando Random Forest
for column in columns_with_nulls:
    is_classification = null_columns_types[column] == 'object'
    x1 = impute_with_random_forest_encoded(x1, column, is_classification)

# Verificar si quedan valores nulos
remaining_nulls = x1.isnull().sum().sum()

In [25]:
x1.isnull().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 80, dtype: int64

# Utilizar esta data para analizar el test (x2)